In [ ]:
import os
import sys

import pandas as pd

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

%load_ext autoreload
%autoreload 2

from utils.utils import (
    generate_data,
    generate_rejections,
    generate_roster,
    print_project_summary,
    process_applications,
    student_assignment,
)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [305]:
file_location = "../data/Spring 25 Matching_mini.xlsx"
prioritized_students = []#"kbarbarossa@uchicago.edu"]
deprioritized_students = []#"hsjpaul@uchicago.edu","wanniyang@uchicago.edu","nalinb@uchicago.edu"]
technical_project_list = []#"Argonne", "Center for Living Systems", "CMAP", "Satellite-Based Detection of Ancient Water Systems", "SPUN"]
drop_project_list = []#"Satellite-Based Detection of Ancient Water Systems", "SPUN"]

application_df, forced_assignments = process_applications(file_location, deprioritized_students, prioritized_students)

data_to_process_dict = generate_data(application_df, technical_project_list)

priority_weights = [5, 10, 15, 20]
max_students = {}
all_project_list = data_to_process_dict.pop("all_project_list")

assignment_df = student_assignment(priority_weights = priority_weights, **data_to_process_dict,
                                   max_students_dict=max_students, preassigned_students=forced_assignments,
                                   number_of_projects_to_run=None, drop_projects=drop_project_list, verbose=False)
                                   
print(f"\nAverage Ranking: {assignment_df.Ranking.mean():.2f}")
print(assignment_df.Ranking.value_counts())
print_project_summary(assignment_df, all_project_list)

assignment_df.to_csv("../data/assignments.csv", index=False)


Dropping 0 students because they do not have a computer and not required
Total Students available for matching: 10
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/hannifan/projects/the-clinic/.venv/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/sr/lrm99gp93rx47xspcx2z49gr0000gp/T/374543ce993f49edaa46e263241d2f4a-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/sr/lrm99gp93rx47xspcx2z49gr0000gp/T/374543ce993f49edaa46e263241d2f4a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 64 COLUMNS
At line 659 RHS
At line 719 BOUNDS
At line 810 ENDATA
Problem MODEL has 59 rows, 90 columns and 342 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.01


Average Rankin

,Project Name,Number,High Priority,High-Med Priority,Med Priority,Low Priority,Experienced,Rankings,Average Ranking
0,Argonne,3,1,2,0,0,1,"1,100,100",67.0
1,CMAP,1,0,0,1,0,1,0,0.0
2,Kids First Chicago,1,1,0,0,0,1,0,0.0
3,UChicago Library,1,0,0,0,1,0,100,100.0
4,Center for Living Systems,0,0,0,0,0,0,,0.0
5,College Financial Health,0,0,0,0,0,0,,0.0
6,RAFI - Grocery,0,0,0,0,0,0,,0.0
7,DSI - HPC,0,0,0,0,0,0,,0.0
8,Groundwork Bridgeport,0,0,0,0,0,0,,0.0
9,Satellite-Based Detection of Ancient Water Sys...,0,0,0,0,0,0,,0.0


In [288]:
# Generate roster for matching spreadsheet
roster = generate_roster(application_df, assignment_df)
roster.to_csv("../data/roster.csv", index=False)
print(roster.shape)
roster.head()

(85, 8)


,Project,Name,GitHub,Email,Chicago ID,Degree Program,Concentration,Returning
0,Argonne,Uday Malik,udaymalik12,umalik@uchicago.edu,41080320C,Undergrad: 4th year,Data Science,1
1,Argonne,Xinwen Huang,Hwx-w,xinwenh@uchicago.edu,60823345V,MA or MS 2nd year,MA Public Policy (MPP),0
2,Argonne,Ye Joon Han,yejoonhan07,yejoonh@uchicago.edu,20433786D,MA or MS 1st year,Data Science,0
3,BDC,Isabella Mercado,imercadouchicago,imercado@uchicago.edu,12317003,Undergrad: 4th year,Data Science,1
4,BDC,Jerald Shu,j-shu-uofc,jeraldshu@uchicago.edu,01983809J,Undergrad: 4th year,Data Science,1


In [289]:
print(roster.shape[0])

85


In [290]:
print(roster["Email"].unique().shape)

(85,)


In [291]:
# Generate rejection csv
rejections = generate_rejections(assignment_df)
rejections.to_csv("../data/rejections.csv", index=False)
print(rejections.shape)
rejections.head()


(7, 1)


,Email
0,beichiwu@uchicago.edu
1,gdiaz0618@uchicago.edu
2,griffin13@uchicago.edu
3,jycchien@uchicago.edu
4,leahdimsu@uchicago.edu


In [292]:
assert roster.shape[0] + rejections.shape[0] == application_df.shape[0], 'Mismatched size'

In [293]:
project_to_comp = "Kids First Chicago"

list_of_not_assigned = list(assignment_df.loc[(assignment_df.Ranking.isna()), "Email Address"])
prefs_df = data_to_process_dict["ranking"].copy()
unmatched_prefs = prefs_df.loc[(prefs_df.loc[:, "Project Name"] == project_to_comp) & prefs_df.loc[:, "Email Address"].isin(list_of_not_assigned)
             & (prefs_df.Ranking < 100), :]

unmatched_info = pd.merge(unmatched_prefs, assignment_df.loc[:, ["Email Address", "Experienced", "Priority"]], how="left", on="Email Address")

matched = assignment_df.loc[(assignment_df.loc[:, "Project Assigned"] == project_to_comp), :]
display(matched.sort_values("Ranking"))

display(unmatched_info)

,Email Address,Priority,Experienced,Project Assigned,Ranking
18,ddanaie@uchicago.edu,1,False,Kids First Chicago,0
47,kbarbarossa@uchicago.edu,3,False,Kids First Chicago,0
64,rdrodriguez@uchicago.edu,1,True,Kids First Chicago,0
79,wanniyang@uchicago.edu,2,False,Kids First Chicago,0


,Email Address,Project Name,Ranking,Experienced,Priority
0,lstarnes@uchicago.edu,Kids First Chicago,5.0,True,1


In [294]:
# Priority students that were not assigned to a project
assignment_df[(assignment_df["Priority"] == 1) & (assignment_df["Project Assigned"].isna())]

,Email Address,Priority,Experienced,Project Assigned,Ranking
53,lstarnes@uchicago.edu,1,True,None,None


In [295]:
# Non-priority students that were assigned to a project
assignment_df[(assignment_df["Project Assigned"].notna()) & (assignment_df["Priority"] > 1)]

,Email Address,Priority,Experienced,Project Assigned,Ranking
2,akshar@uchicago.edu,2,False,Groundwork Bridgeport,1.0
4,arkadeep@uchicago.edu,2,False,RAFI - Grocery,1.0
7,betsyshi@uchicago.edu,2,False,DSI - Agents,1.0
8,caitlinp@uchicago.edu,2,True,Data and Democracy,1.0
10,ceden26@uchicago.edu,3,False,Rwanda - Climate,0
11,changge@uchicago.edu,2,False,DSI - HPC,1.0
12,charliewang436@uchicago.edu,3,False,Internet Equity Initiative,1.0
14,cyang19@uchicago.edu,2,False,Climate Cabinet,1.0
15,daniel.li3002@gmail.com,4,True,SPUN,2.0
20,dkeng@uchicago.edu,4,False,Satellite-Based Detection of Ancient Water Sys...,1.0


In [296]:
n_return = assignment_df[(assignment_df["Project Assigned"].notna()) & (assignment_df["Ranking"] == 0)].shape[0]
n_priority = assignment_df[(assignment_df["Priority"] == 1)].shape[0]
n_placed = assignment_df[(assignment_df["Project Assigned"].notna())].shape[0]

print(f"{n_return} returning students")
print(f"{n_priority} priority students")
print(f"{n_placed} total placed students")


39 returning students
36 priority students
85 total placed students


In [297]:
# All students that were assigned
assignment_df[(assignment_df["Project Assigned"].notna())][['Email Address', 'Project Assigned']]

,Email Address,Project Assigned
0,acherman@uchicago.edu,CMAP
1,akim03@uchicago.edu,IDI
2,akshar@uchicago.edu,Groundwork Bridgeport
3,alexmosher@uchicago.edu,Invenergy
4,arkadeep@uchicago.edu,RAFI - Grocery
...,...,...
87,yufeil@uchicago.edu,Internet Equity Initiative
88,yurouli@uchicago.edu,FS6
89,zachlaurence@uchicago.edu,CLS
90,zekaishen@uchicago.edu,Satellite-Based Detection of Ancient Water Sys...


In [257]:
# Check specific student(s) by email
assignment_df[assignment_df["Email Address"].isin(["wanniyang@uchicago.edu", "kbarbarossa@uchicago.edu"])]

,Email Address,Priority,Experienced,Project Assigned,Ranking
47,kbarbarossa@uchicago.edu,3,False,Kids First Chicago,0
79,wanniyang@uchicago.edu,2,False,Kids First Chicago,0
